# Khai báo thư viện

In [ ]:
%pip install langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.0 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.5/757.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.2/582.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: packaging━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/26 [python-dotenv]
    Found existing installation: packaging 23.0━━━━━━━━━━━━━━━  3/26 [python-dotenv]
    Uninstalling packaging-23.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/26 [python-dotenv]
      Successfully uninstalled packaging-23.0━━━━━━━━━━━━━━━━━  3/26 [python-dotenv]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26/26 [langchain-communi

In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import io
import time
from datetime import datetime
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
import os
import re
import json
from pathlib import Path  # tiện hơn os.path
from underthesea import sent_tokenize
from transformers import AutoTokenizer
import faiss
import shutil

/home/tesla/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tesla/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Load GPU

In [2]:
import torch

print(f"Số lượng GPU: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(
        f"  Tổng VRAM: {torch.cuda.get_device_properties(i).total_memory / 1024 / 1024**2:.2f} GB"
    )

device = torch.device("cuda:0")  # Chỉ định GPU 0 (Tesla T4)
print(f"\nĐang sử dụng: {torch.cuda.get_device_name(device)}")

Số lượng GPU: 2
GPU 0: Tesla T4
  Tổng VRAM: 14.57 GB
GPU 1: NVIDIA GeForce GT 1030
  Tổng VRAM: 1.94 GB

Đang sử dụng: Tesla T4


In [5]:
!nvidia-smi

Thu Jul 17 20:19:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GT 1030         Off |   00000000:18:00.0 Off |                  N/A |
| 35%   35C    P8             N/A /   30W |     477MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load model embedding

In [4]:
# Load model từ ổ cứng
model_path = "./Vietnamese_Embedding"
model = SentenceTransformer(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Tạo embeddings từ PDF

In [3]:
# Đường dẫn file PDF bạn muốn xử lý
pdf_path = "../data/file/CTKM_01.pdf"
output_dir = "./results"

all_faiss_path = os.path.join(output_dir, "all_faiss.index")
all_pickle_path = os.path.join(output_dir, "all_embeddings.pkl")

## Bước 1: Check xem file pdf đó đã được embedding chưa

In [32]:
def is_pdf_embedded(path):
    """
    Kiểm tra xem file PDF đã được embedding hay chưa,
    dựa vào all_embeddings.pkl (danh sách các file đã xử lý).
    """
    if not os.path.exists(all_pickle_path):
        return False  # Chưa có dữ liệu chung => chắc chắn chưa nhúng gì

    pdf_name = os.path.splitext(os.path.basename(path))[0]

    with open(all_pickle_path, 'rb') as f:
        all_data = pickle.load(f)

    existing_pdf_names = {entry['pdf_name'] for entry in all_data}

    return pdf_name in existing_pdf_names

In [33]:
if is_pdf_embedded(pdf_path):
    print("📌 PDF này đã được embedding trước đó.")
else:
    print("🔄 PDF này chưa được embedding.")

📌 PDF này đã được embedding trước đó.


## Bước 2: OCR PDF

In [6]:
def preprocess_image(img):
    """
    Tiền xử lý ảnh để cải thiện OCR
    """
    if img.mode != 'L':
        img = img.convert('L')

    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(1.5)
    img = img.filter(ImageFilter.SHARPEN)

    return img

In [ ]:
def ocr_pdf_to_text(pdf_path, output_dir):
    """
    OCR file PDF thành text
    """
    try:
        print(f"📖 Đang OCR file: {pdf_path}")
        doc = fitz.open(pdf_path)
        total_pages = len(doc)

        full_text = ""
        ocr_config = r'--oem 3 --psm 6 -l vie'

        for page_num in range(total_pages):
            print(f"🔄 Xử lý trang {page_num + 1}/{total_pages}...")

            page = doc.load_page(page_num)
            matrix = fitz.Matrix(2.5, 2.5)
            pix = page.get_pixmap(matrix=matrix)
            img_data = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_data))

            img = preprocess_image(img)

            try:
                page_text = pytesseract.image_to_string(img, config=ocr_config)
                full_text += page_text.strip()

            except Exception as e:
                print(f"   ❌ Lỗi OCR trang {page_num + 1}: {e}")

        doc.close()
        print(f"✅ Hoàn thành OCR {total_pages} trang")

        # 🧠 Tạo tên file JSON theo tên file PDF
        pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_path = os.path.join(output_dir, pdf_name, f"{pdf_name}_ocr.txt")

        # Tạo thư mục nếu chưa tồn tại
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Lưu file JSON
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(full_text)

        print(f"📄 Kết quả đã lưu vào: {output_path}")

        # Trả về danh sách các trang với nội dung
        return full_text

    except Exception as e:
        print(f"❌ Lỗi OCR: {e}")
        return None

In [8]:
# Bước 1: OCR PDF
raw_text = ocr_pdf_to_text(pdf_path, output_dir)
if not raw_text:
    print("❌ Không thể OCR file PDF. Vui lòng kiểm tra lại.")
    exit(1)

📖 Đang OCR file: ../data/file/TTTS2025.pdf
🔄 Xử lý trang 1/22...
🔄 Xử lý trang 2/22...
🔄 Xử lý trang 3/22...
🔄 Xử lý trang 4/22...
🔄 Xử lý trang 5/22...
🔄 Xử lý trang 6/22...
🔄 Xử lý trang 7/22...
🔄 Xử lý trang 8/22...
🔄 Xử lý trang 9/22...
🔄 Xử lý trang 10/22...
🔄 Xử lý trang 11/22...
🔄 Xử lý trang 12/22...
🔄 Xử lý trang 13/22...
🔄 Xử lý trang 14/22...
🔄 Xử lý trang 15/22...
🔄 Xử lý trang 16/22...
🔄 Xử lý trang 17/22...
🔄 Xử lý trang 18/22...
🔄 Xử lý trang 19/22...
🔄 Xử lý trang 20/22...
🔄 Xử lý trang 21/22...
🔄 Xử lý trang 22/22...
✅ Hoàn thành OCR 22 trang
📄 Kết quả đã lưu vào: ./results/TTTS2025/TTTS2025_ocr.txt


## Bước 3: Làm sạch text

In [ ]:
def clean_text(text, pdf_path, output_dir):
    """
    Làm sạch một đoạn văn bản OCR (string)
    """
    # Loại ký tự không mong muốn (giữ lại tiếng Việt, toán học, đơn vị)
    text = re.sub(r'[^\w\s.,;:()\[\]?!\"\'\-–—…°%‰≥≤→←≠=+/*<>\n\r]', '', text)

    # Xử lý lỗi xuống dòng giữa từ hoặc giữa câu
    text = re.sub(r'-\n', '', text)             # nối từ bị gạch nối xuống dòng
    text = re.sub(r'\n(?=\w)', ' ', text)       # dòng xuống không hợp lý → nối câu

    # Dấu chấm lặp vô nghĩa → ba chấm
    text = re.sub(r'\.{3,}', '...', text)

    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\n\s*\n', '\n\n', text)   # giữ ngắt đoạn
    text = re.sub(r'[ \t]+', ' ', text)       # nhiều khoảng trắng → 1 dấu cách
    text = re.sub(r' *\n *', '\n', text)      # bỏ khoảng trắng đầu/cuối dòng

    # Lưu file
    clean_text = text.strip()
    
    # 🧠 Tạo tên file JSON theo tên file PDF
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_path = os.path.join(output_dir, pdf_name, f"{pdf_name}_clean.txt")

    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Lưu file JSON
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(clean_text)

    print(f"📄 Kết quả đã lưu vào: {output_path}")
    
    return clean_text


In [11]:
# Bước 2: Làm sạch text
print("🧹 Làm sạch text...")
cleaned_text = clean_text(raw_text, pdf_path, output_dir)

🧹 Làm sạch text...
📄 Kết quả đã lưu vào: ./results/TTTS2025/TTTS2025_clean.txt


## Bước 4: Chia thành chunks

In [12]:
def split_sections(text):
    """
    Tách text thành các phần theo tiêu đề kiểu I., 1., a)
    """
    sections = re.split(r'\n(?=(?:[IVXLCDM]+\.)|(?:\d+\.)|(?:[a-z]\)))', text)
    return [s.strip() for s in sections if s.strip()]

In [13]:
def split_text_to_chunks_vi_tokenized_with_section(text, chunk_size=512, overlap=50):
    """
    Chia văn bản tiếng Việt thành các chunk dựa trên số token,
    giữ nguyên cấu trúc section và câu.
    """
    sections = split_sections(text)
    all_chunks = []

    for section in sections:
        sentences = sent_tokenize(section)
        current_chunk = []
        current_tokens = 0

        for sentence in sentences:
            num_tokens = len(tokenizer.tokenize(sentence))

            if current_tokens + num_tokens > chunk_size:
                chunk_text = '\n'.join(current_chunk).strip()
                all_chunks.append(chunk_text)

                # Overlap bằng token
                overlap_chunk = []
                total = 0
                for s in reversed(current_chunk):
                    toks = len(tokenizer.tokenize(s))
                    if total + toks > overlap:
                        break
                    overlap_chunk.insert(0, s)
                    total += toks

                current_chunk = overlap_chunk + [sentence]
                current_tokens = total + num_tokens
            else:
                current_chunk.append(sentence)
                current_tokens += num_tokens

        if current_chunk:
            all_chunks.append(' '.join(current_chunk).strip())

    return all_chunks

In [14]:
# Bước 3: Chia thành chunks
print("✂️ Chia text thành chunks...")
chunks = split_text_to_chunks_vi_tokenized_with_section(cleaned_text)
print(f"📝 Đã tạo {len(chunks)} chunks")

✂️ Chia text thành chunks...


📝 Đã tạo 56 chunks


## Bước 5: Tạo embeddings

In [15]:
def create_embeddings(chunks):
    """
    Tạo embeddings cho các text chunks
    """
    try:
        print(f"🔄 Tạo embeddings cho {len(chunks)} chunks...")
        embeddings = model.encode(chunks, show_progress_bar=True)

        print(f"✅ Hoàn thành tạo embeddings")
        return embeddings

    except Exception as e:
        print(f"❌ Lỗi tạo embeddings: {e}")
        return None

In [16]:
# Bước 4: Tạo embeddings
embeddings = create_embeddings(chunks)
if embeddings is None:
    print("❌ Không thể tạo embeddings. Vui lòng kiểm tra lại.")
    exit(1)

🔄 Tạo embeddings cho 56 chunks...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Hoàn thành tạo embeddings


## Bước 6: Lưu embeddings

In [ ]:
def save_embeddings(chunks, embeddings, pdf_path, output_dir):
    """
    Lưu embeddings và chunks vào file
    """
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]

    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(os.path.join(output_dir, pdf_name), exist_ok=True)

    # Lưu dữ liệu
    data = {
        'pdf_name': pdf_name,
        'chunks': chunks,
        'embeddings': embeddings,
        'created_at': datetime.now().isoformat()
    }

    # Lưu embeddings (pickle)
    pickle_path = os.path.join(output_dir, pdf_name, f"{pdf_name}_embeddings.pkl")
    with open(pickle_path, 'wb') as f:
        pickle.dump(data, f)

    # Lưu chunks (text file)
    chunks_path = os.path.join(output_dir, pdf_name, f"{pdf_name}_chunks.txt")
    with open(chunks_path, 'w', encoding='utf-8') as f:
        f.write(f"CHUNKS TỪ FILE: {pdf_name}.pdf\n")
        f.write(f"Tạo lúc: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Tổng số chunks: {len(chunks)}\n")
        f.write("=" * 60 + "\n\n")

        for i, chunk in enumerate(chunks, 1):
            f.write(f"CHUNK {i}:\n")
            f.write("-" * 30 + "\n")
            f.write(chunk + "\n")
            f.write("-" * 30 + "\n\n")

    # Lưu thông tin embeddings (text file)
    embedding_info_path = os.path.join(
        output_dir, pdf_name, f"{pdf_name}_embedding_info.txt")
    with open(embedding_info_path, 'w', encoding='utf-8') as f:
        f.write(f"THÔNG TIN EMBEDDINGS: {pdf_name}.pdf\n")
        f.write(f"Tạo lúc: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 60 + "\n\n")

        f.write(f"📊 THỐNG KÊ:\n")
        f.write(f"- Tổng số chunks: {len(chunks)}\n")
        f.write(f"- Kích thước embeddings: {embeddings.shape}\n")
        f.write(f"- Kiểu dữ liệu: {embeddings.dtype}\n")
        f.write(
            f"- Kích thước mỗi vector: {embeddings.shape[1]} dimensions\n\n")

        f.write(f"📝 PREVIEW EMBEDDINGS (5 chunks đầu):\n")
        f.write("-" * 50 + "\n")

        for i in range(min(5, len(chunks))):
            f.write(f"\nCHUNK {i+1}:\n")
            f.write(f"Text: {chunks[i][:100]}...\n")
            f.write(
                f"Embedding vector (10 giá trị đầu): {embeddings[i][:10].tolist()}\n")
            f.write(f"Vector norm: {np.linalg.norm(embeddings[i]):.4f}\n")
            f.write("-" * 30 + "\n")
    
    # 4️⃣ Lưu FAISS index
    index_path = os.path.join(output_dir, pdf_name, f"{pdf_name}_faiss.index")
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.astype(np.float32))
    faiss.write_index(index, index_path)

    # --- 🔁 Cập nhật FAISS chung ---
    all_faiss_path = os.path.join(output_dir, "all_faiss.index")
    if os.path.exists(all_faiss_path):
        index_all = faiss.read_index(all_faiss_path)
    else:
        index_all = faiss.IndexFlatL2(dim)

    index_all.add(embeddings.astype(np.float32))
    faiss.write_index(index_all, all_faiss_path)

    # --- 🔁 Cập nhật pickle chung ---
    all_pickle_path = os.path.join(output_dir, "all_embeddings.pkl")
    if os.path.exists(all_pickle_path):
        with open(all_pickle_path, 'rb') as f:
            all_data = pickle.load(f)
    else:
        all_data = []

    all_data.append(data)

    with open(all_pickle_path, 'wb') as f:
        pickle.dump(all_data, f)

    print(f"💾 Đã lưu embeddings: {pickle_path}")
    print(f"📄 Đã lưu chunks: {chunks_path}")
    print(f"📊 Đã lưu thông tin embeddings: {embedding_info_path}")
    print(f"📌 Đã lưu FAISS index: {index_path}")
    print(f"🔁 Cập nhật FAISS chung: {all_faiss_path}")
    print(f"📦 Cập nhật pickle chung: {all_pickle_path}")

    return pickle_path, index_path

In [21]:
# Bước 5: Lưu embeddings
pickle_path, faiss_path = save_embeddings(chunks, embeddings, pdf_path, output_dir)

print(f"\n🎉 HOÀN THÀNH!")
print(f"📊 Thống kê:")
print(f"   - Số chunks: {len(chunks)}")
print(f"   - Kích thước embedding: {embeddings.shape}")
print(f"   - File embeddings: {pickle_path}")
print(
    f"   - File chunks: {pickle_path.replace('_embeddings.pkl', '_chunks.txt')}")
print(
    f"   - File thông tin: {pickle_path.replace('_embeddings.pkl', '_embedding_info.txt')}")

# Lưu lại đường dẫn file .pkl để sử dụng sau
print(f"✅ Đường dẫn embeddings: {pickle_path}")
print(f"✅ Đường dẫn FAISS index: {faiss_path}")

💾 Đã lưu embeddings: ./results/TTTS2025/TTTS2025_embeddings.pkl
📄 Đã lưu chunks: ./results/TTTS2025/TTTS2025_chunks.txt
📊 Đã lưu thông tin embeddings: ./results/TTTS2025/TTTS2025_embedding_info.txt
📌 Đã lưu FAISS index: ./results/TTTS2025/TTTS2025_faiss.index
🔁 Cập nhật FAISS chung: ./results/all_faiss.index
📦 Cập nhật pickle chung: ./results/all_embeddings.pkl

🎉 HOÀN THÀNH!
📊 Thống kê:
   - Số chunks: 56
   - Kích thước embedding: (56, 1024)
   - File embeddings: ./results/TTTS2025/TTTS2025_embeddings.pkl
   - File chunks: ./results/TTTS2025/TTTS2025_chunks.txt
   - File thông tin: ./results/TTTS2025/TTTS2025_embedding_info.txt
✅ Đường dẫn embeddings: ./results/TTTS2025/TTTS2025_embeddings.pkl
✅ Đường dẫn FAISS index: ./results/TTTS2025/TTTS2025_faiss.index


## Xóa pdf

In [10]:
# Xóa thư mục chứa embeddings của file PDF
def delete_pdf_folder(pdf_path, output_dir):
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    folder_path = os.path.join(output_dir, pdf_name)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"🗑️ Đã xoá thư mục: {folder_path}")
    else:
        print(f"⚠️ Thư mục không tồn tại: {folder_path}")

In [11]:
# Xóa khỏi all_embeddings.pkl
def remove_from_all_embeddings(pdf_path, all_pickle_path):
    if not os.path.exists(all_pickle_path):
        print("⚠️ all_embeddings.pkl không tồn tại.")
        return

    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]

    with open(all_pickle_path, 'rb') as f:
        all_data = pickle.load(f)

    new_data = [entry for entry in all_data if entry['pdf_name'] != pdf_name]

    if len(new_data) == len(all_data):
        print(f"❌ PDF '{pdf_name}' không có trong all_embeddings.pkl.")
    else:
        with open(all_pickle_path, 'wb') as f:
            pickle.dump(new_data, f)
        print(f"✅ Đã xoá '{pdf_name}' khỏi all_embeddings.pkl.")


In [12]:
# Xóa khỏi all_faiss.index
def rebuild_faiss_from_pickle(all_pickle_path, all_faiss_path):
    if not os.path.exists(all_pickle_path):
        print("⚠️ all_embeddings.pkl không tồn tại.")
        return

    with open(all_pickle_path, 'rb') as f:
        all_data = pickle.load(f)

    if not all_data:
        print("⚠️ Không còn dữ liệu nào trong all_embeddings.pkl.")
        if os.path.exists(all_faiss_path):
            os.remove(all_faiss_path)
            print("🗑️ Đã xoá all_faiss.index rỗng.")
        return

    # Gom tất cả vectors lại
    all_vectors = np.vstack([entry['embeddings'] for entry in all_data]).astype(np.float32)
    dim = all_vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(all_vectors)
    faiss.write_index(index, all_faiss_path)
    print(f"🔄 Đã xây lại all_faiss.index với {all_vectors.shape[0]} vectors.")

In [ ]:
def delete_pdf_folder_and_update(pdf_path, output_dir):
    all_pickle_path = os.path.join(output_dir, "all_embeddings.pkl")
    all_faiss_path = os.path.join(output_dir, "all_faiss.index")
    
    delete_pdf_folder(pdf_path, output_dir)
    remove_from_all_embeddings(pdf_path, all_pickle_path)
    rebuild_faiss_from_pickle(all_pickle_path, all_faiss_path)

In [19]:
delete_pdf_folder_and_update(pdf_path, output_dir)

✅ Đã xoá 'CTKM_01' khỏi all_embeddings.pkl.
🔄 Đã xây lại all_faiss.index với 56 vectors.


# Demo tìm kiếm với câu hỏi bất kỳ

In [43]:
# Thử tìm kiếm với câu hỏi bất kỳ
query = "Trụ sở của trường nằm ở đâu?"

## Dùng Pickle

## Bước 1: Load embedding

In [23]:
def load_embeddings(embeddings_path):
    """
    Tải embeddings từ file
    """
    try:
        with open(embeddings_path, 'rb') as f:
            data = pickle.load(f)

        print(f"📂 Đã tải embeddings từ: {embeddings_path}")
        return data

    except Exception as e:
        print(f"❌ Lỗi tải embeddings: {e}")
        return None

In [25]:
# Tải lại dữ liệu embeddings đã tạo
data = load_embeddings(pickle_path)

📂 Đã tải embeddings từ: ./results/TTTS2025_embeddings.pkl


## Bước 2: Tìm kiếm

In [26]:
def find_similar_chunks(query, embeddings_data, top_k=3):
    """
    Tìm các chunk tương tự với query
    """
    try:
        # Tạo embedding cho query
        query_embedding = model.encode([query])

        # Tính cosine similarity
        similarities = np.dot(
            embeddings_data['embeddings'], query_embedding.T).flatten()

        # Lấy top_k kết quả
        top_indices = np.argsort(similarities)[-top_k:][::-1]

        results = []
        for idx in top_indices:
            results.append({
                'chunk': embeddings_data['chunks'][idx],
                'similarity': similarities[idx],
                'index': idx
            })

        return results

    except Exception as e:
        print(f"❌ Lỗi tìm kiếm: {e}")
        return []

In [44]:
results = find_similar_chunks(query, data, top_k=3)

print(f"\n🔍 DEMO TÌM KIẾM")
print(f"📚 PDF: {data['pdf_name']}")
print(f"📝 Số chunks: {len(data['chunks'])}")
print("=" * 50)

# In kết quả
for i, result in enumerate(results, 1):
    print(f"\n{i}. 🔍 Độ tương đồng: {result['similarity']:.4f}")
    print(f"📄 Nội dung: {result['chunk'][:300]}...")



🔍 DEMO TÌM KIẾM
📚 PDF: TTTS2025
📝 Số chunks: 56

1. 🔍 Độ tương đồng: 0.3401
📄 Nội dung: + Tháng 01 năm 2008, Học viện KTQS được Nhà nước công nhận là một trong 15 trường Đại học trọng điêm Quốc gia.
b) Sứ mệnh Sứ mệnh của.
Học viện KTQS là đào tạo nguồn nhân lực chất lượng cao, nghiên cứu phát triên, sản xuât chê thử, chuyên giao công nghệ tiên tiên và hội nhập quốc tê, góp phân đắc lự...

2. 🔍 Độ tương đồng: 0.2782
📄 Nội dung: 1. Tên cơ sở đào tạo: Học viện Kỹ thuật quân sự (MIilitary Technical Academy), tên dân sự là Trường Đại học Kỹ thuật Lê Quý Đôn (Le Quy Đon Technical University)...

3. 🔍 Độ tương đồng: 0.2573
📄 Nội dung: Số điện thoại liên hệ: 069.5 15.2263 6. Địa chỉ công khai quy chế tuyển sinh, thông tin tuyển sinh: http:/www.mta.edu.vn; 7. Các thông tin công khai về hoạt động của Học viện: Địa chỉ công khai thông tin: http://www.mta.edu.vn; 4) Ngành đào tạo Học viện tổ chức đảo tạo trình độ đại học với 15 ngành ...


## Dùng FAISS

## Bước 1: Load FAISS index

In [30]:
def load_faiss_index(index_path):
    print(f"📂 Đang tải FAISS index từ: {index_path}")
    return faiss.read_index(index_path)

In [31]:
index = load_faiss_index(faiss_path)

📂 Đang tải FAISS index từ: ./results/TTTS2025_faiss.index


## Bước 2: Encode câu hỏi thành vector

In [32]:
def encode_query(query):
    return model.encode([query])  # trả về numpy array (1, dim)

In [45]:
query_vector = encode_query(query)

## Bước 3: Truy vấn FAISS

In [48]:
def search_faiss(index, query_vector, top_k=5):
    D, I = index.search(query_vector.astype('float32'), top_k)
    return I[0], D[0]  # indices, distances

In [ ]:
top_k = 3
indices, distances = search_faiss(index, query_vector, top_k)

In [50]:
print("📌 Kết quả truy vấn:")
for i, (idx, dist) in enumerate(zip(indices, distances), 1):
    print(f"\n🔹 Kết quả {i}: (score: {dist:.4f})")
    print(chunks[idx][:500])  # in tối đa 500 ký tự

📌 Kết quả truy vấn:

🔹 Kết quả 1: (score: 1.3199)
+ Tháng 01 năm 2008, Học viện KTQS được Nhà nước công nhận là một trong 15 trường Đại học trọng điêm Quốc gia.
b) Sứ mệnh Sứ mệnh của.
Học viện KTQS là đào tạo nguồn nhân lực chất lượng cao, nghiên cứu phát triên, sản xuât chê thử, chuyên giao công nghệ tiên tiên và hội nhập quốc tê, góp phân đắc lực vào sự nghiệp xây dựng và bảo vệ Tổ quốc, phát triên ngành khoa học công nghệ quân sự Việt Nam.
c) Tâm nhìn
_ Đến năm 2030, Học viện KTQS trở thành trường Đại học nghiên cứu năm trong tốp đâu về Kho

🔹 Kết quả 2: (score: 1.4436)
1. Tên cơ sở đào tạo: Học viện Kỹ thuật quân sự (MIilitary Technical Academy), tên dân sự là Trường Đại học Kỹ thuật Lê Quý Đôn (Le Quy Đon Technical University)

🔹 Kết quả 3: (score: 1.4854)
Số điện thoại liên hệ: 069.5 15.2263 6. Địa chỉ công khai quy chế tuyển sinh, thông tin tuyển sinh: http:/www.mta.edu.vn; 7. Các thông tin công khai về hoạt động của Học viện: Địa chỉ công khai thông tin: http://www.mta.edu.vn;